# What is an Index in Pinecone?

## 1) An index in Pinecone is a database for vector embeddings.
## 2) Imagine you have many items (like documents, images, or text).
## 3) You convert these into vectors (number lists that represent their meaning).
## 4) You put all those vectors into a Pinecone index.
## 5) Later, you can search the index to find similar items — very fast.

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
api_key=os.getenv("PINECONE_API_KEY")

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [6]:
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pincone"

## initialize the pincone client
pc=Pinecone(api_key=api_key)

#create the index 
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud="aws",region="us-east-1")
    )

In [7]:
index=pc.Index(index_name)

index

# Vector Embedding and Sparse matrix

In [9]:
import os 
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embedings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embedings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [11]:
sentences=[
    "In 2023,I visited Paris",
    "In 2022,I visited New York",
    "In 2021,I visited New Orleans",
]

# applying TF-IDF values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

## load to your BM25Encoder object
bm25_encoder=BM25Encoder().load("bm25_values.json")

  0%|          | 0/3 [00:00<?, ?it/s]

# creating retriever

In [14]:
# Which supports both semantic search and syntactic search
retriever=PineconeHybridSearchRetriever(
    embeddings=embedings,
    sparse_encoder=bm25_encoder,
    index=index
)

retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000013C91299E40>, index=<pinecone.db_data.index.Index object at 0x0000013CC3625420>)

In [15]:
retriever.add_texts(
    [
    "In 2023,I visited Paris",
    "In 2022,I visited New York",
    "In 2021,I visited New Orleans",
]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
retriever.invoke("which city did i visited last time")

[Document(metadata={'score': 0.26056844}, page_content='In 2021,I visited New Orleans'),
 Document(metadata={'score': 0.245537102}, page_content='In 2022,I visited New York'),
 Document(metadata={'score': 0.237763956}, page_content='In 2023,I visited Paris')]